<a href="https://www.kaggle.com/code/luisgasparcordeiro/player-stats-by-season?scriptVersionId=115304313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/player-scores/players.csv
/kaggle/input/player-scores/competitions.csv
/kaggle/input/player-scores/games.csv
/kaggle/input/player-scores/game_events.csv
/kaggle/input/player-scores/club_games.csv
/kaggle/input/player-scores/player_valuations.csv
/kaggle/input/player-scores/appearances.csv
/kaggle/input/player-scores/clubs.csv


In [2]:
# Read CSVs into dataframes
games = pd.read_csv("/kaggle/input/player-scores/games.csv")
appearances = pd.read_csv("/kaggle/input/player-scores/appearances.csv")

# 1 - Player Stats
Let's define a function that return player's key stats for a given season

In [3]:
def player_stats(player_name, season):
    
    df = appearances.merge(games, on=['game_id'], how='left')
    df = df[df['player_pretty_name'] == player_name]
    df = df[df['season'] == season]
    
    if (df.shape[0] == 0):
        Out = [(np.nan,player_name, season,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','player_pretty_names','season','goals','games',
                                                     'assists','minutes_played','goals_for','goals_against','clean_sheet'])
        return out_df
    
    else:
        
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
                      else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
                      else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
                      else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
                      else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
                      else 0 if row['goals_against'] > 0
                      else np.nan, axis=1)
        
        df = df.groupby(['player_id',"player_pretty_name","season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                      'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                      'goals_against' : 'sum', 'clean_sheet' : 'sum'})
        out_df = df.rename(columns={'game_id': 'games'})

        return out_df

In [4]:
player_stats('Cristiano Ronaldo', 2019)

,player_id,player_pretty_name,season,goals,games,assists,minutes_played,goals_for,goals_against,clean_sheet
0,8198,Cristiano Ronaldo,2019.0,35,45,7,3991,86.0,51.0,15


In [5]:
player_stats('Manuel Neuer', 2021)

,player_id,player_pretty_name,season,goals,games,assists,minutes_played,goals_for,goals_against,clean_sheet
0,17259,Manuel Neuer,2021.0,0,40,1,3591,125.0,41.0,14


## Looks Good!